In [ ]:
import zipfile
import os

def unzip_folder(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Đã giải nén {zip_path} vào thư mục {extract_to}")


zip_path = '/content/GOOG.zip'
extract_to = 'GOOG_image'

unzip_folder(zip_path, extract_to)

Đã giải nén /content/GOOG.zip vào thư mục GOOG_image


In [ ]:
import pandas as pd
import numpy as np
import torch
import open_clip
from PIL import Image
from sklearn.preprocessing import StandardScaler
import os
from datetime import datetime

def load_images_in_order(image_dir, image_names_sorted):
    images = []
    for name in image_names_sorted:
        path = os.path.join(image_dir, f"{name}.png")
        img = Image.open(path).convert("RGB")
        images.append(img)
    return images

In [ ]:


def main():
    csv_path = "/content/sp500_industry.csv"
    image_dir = "/content/GOOG_image/GOOG"
    seq_len = 96
    pred_len = 24
    window_size = seq_len + pred_len
    top_k = 20

    df = pd.read_csv(csv_path)
    df = df[df['Symbol'] == 'GOOG']
    df = df[['Date', 'Close']]
    df['Date'] = pd.to_datetime(df['Date'], utc=True)
    df = df.sort_values(by='Date')

    values = df['Close'].values.astype(np.float32)
    scaler = StandardScaler()
    scaled = scaler.fit_transform(values.reshape(-1, 1)).flatten()

    full_windows = np.array([
        scaled[i:i+window_size] for i in range(len(scaled) - window_size + 1)
    ])  # (N, 120)
    xH_windows = full_windows[:, :seq_len]  # (N, 96)

    #  Map windows to image filenames
    dates = df['Date'].values
    date_end_indices = [
        dates[i + seq_len - 1].astype('datetime64[D]').astype(str).replace("-", "")
        for i in range(len(scaled) - window_size + 1)
    ]

    # Load and preprocess images
    images = load_images_in_order(image_dir, date_end_indices)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, _, preprocess = open_clip.create_model_and_transforms(
        'ViT-B-32', pretrained='laion2b_s34b_b79k', device=device
    )
    model.eval()

    print("Encoding images with OpenCLIP...")
    image_tensors = torch.stack([preprocess(img) for img in images]).to(device)
    with torch.no_grad():
        image_embeddings = model.encode_image(image_tensors)
        image_embeddings = image_embeddings / image_embeddings.norm(dim=-1, keepdim=True)
    image_embeddings = image_embeddings.cpu().numpy()

    #  Retrieval
    print("Performing retrieval...")
    reference_indices = []
    for i in range(len(image_embeddings)):
        query = image_embeddings[i]
        sims = image_embeddings @ query
        sims[i] = -np.inf
        topk = np.argsort(sims)[-top_k:][::-1]
        reference_indices.extend(topk.tolist())

    reference_tensor = torch.tensor(reference_indices, dtype=torch.long)
    reference_tensor = torch.clamp(reference_tensor, min=0, max=len(full_windows)-1)
    print(reference_tensor)
    os.makedirs("retrieval_output", exist_ok=True)
    torch.save(reference_tensor, "retrieval_output/reference_only_96_openclip_20_1.pt")
    print(" Saved to retrieval_output/reference_only_96_openclip.pt with shape:", reference_tensor.shape)

if __name__ == "__main__":
    main()


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1123,) + inhomogeneous part.

In [ ]:
!pip install open-clip-torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 803.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


# experiment top K & step_size

In [ ]:
def main():
    csv_path = "/content/sp500_industry.csv"
    image_dir = "/content/GOOG_image/GOOG"
    seq_len = 96
    pred_len = 24
    window_size = seq_len + pred_len

    top_k_list = [5, 10, 20]
    step_sizes = [1, 5, 10]

    df = pd.read_csv(csv_path)
    df = df[df['Symbol'] == 'GOOG']
    df = df[['Date', 'Close']]
    df['Date'] = pd.to_datetime(df['Date'], utc=True)
    df = df.sort_values(by='Date')

    values = df['Close'].values.astype(np.float32)
    scaler = StandardScaler()
    scaled = scaler.fit_transform(values.reshape(-1, 1)).flatten()


    dates = df['Date'].values

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, _, preprocess = open_clip.create_model_and_transforms(
        'ViT-B-32', pretrained='laion2b_s34b_b79k', device=device
    )
    model.eval()

    for step_size in step_sizes:
        print(f" Step size = {step_size}")

        full_windows = np.array([
            scaled[i:i+window_size] for i in range(0, len(scaled) - window_size + 1, step_size)
        ])  # (N, 120)
        xH_windows = full_windows[:, :seq_len]

        date_end_indices = [
            dates[i + seq_len - 1].astype('datetime64[D]').astype(str).replace("-", "")
            for i in range(0, len(scaled) - window_size + 1, step_size)
        ]

        # Load and preprocess images
        images = load_images_in_order(image_dir, date_end_indices)
        image_tensors = torch.stack([preprocess(img) for img in images]).to(device)
        with torch.no_grad():
            image_embeddings = model.encode_image(image_tensors)
            image_embeddings = image_embeddings / image_embeddings.norm(dim=-1, keepdim=True)
        image_embeddings = image_embeddings.cpu().numpy()

        for top_k in top_k_list:
            print(f" Retrieval with top_k = {top_k}")

            reference_indices = []
            for i in range(len(image_embeddings)):
                query = image_embeddings[i]
                sims = image_embeddings @ query
                sims[i] = -np.inf
                topk = np.argsort(sims)[-top_k:][::-1]
                reference_indices.extend(topk.tolist())

            reference_tensor = torch.tensor(reference_indices, dtype=torch.long)
            reference_tensor = torch.clamp(reference_tensor, min=0, max=len(full_windows)-1)

            # Save to file
            filename = f"retrieval_output/reference_only_96_openclip_{top_k}_{step_size}.pt"
            os.makedirs("retrieval_output", exist_ok=True)
            torch.save(reference_tensor, filename)
            print(f" {filename} with shape {reference_tensor.shape}")
if __name__ == "__main__":
    main()

 Step size = 1
 Retrieval with top_k = 5
 retrieval_output/reference_only_96_openclip_5_1.pt with shape torch.Size([5595])
 Retrieval with top_k = 10
 retrieval_output/reference_only_96_openclip_10_1.pt with shape torch.Size([11190])
 Retrieval with top_k = 20
 retrieval_output/reference_only_96_openclip_20_1.pt with shape torch.Size([22380])
 Step size = 5
 Retrieval with top_k = 5
 retrieval_output/reference_only_96_openclip_5_5.pt with shape torch.Size([1120])
 Retrieval with top_k = 10
 retrieval_output/reference_only_96_openclip_10_5.pt with shape torch.Size([2240])
 Retrieval with top_k = 20
 retrieval_output/reference_only_96_openclip_20_5.pt with shape torch.Size([4480])
 Step size = 10
 Retrieval with top_k = 5
 retrieval_output/reference_only_96_openclip_5_10.pt with shape torch.Size([560])
 Retrieval with top_k = 10
 retrieval_output/reference_only_96_openclip_10_10.pt with shape torch.Size([1120])
 Retrieval with top_k = 20
 retrieval_output/reference_only_96_openclip_20_10